In [1]:
import dask
dask.config.config

# persist // compute .  pins to memory and future computation can use this result rather than re-compute
# lazy execution 

{'temporary-directory': None,
 'dataframe': {'shuffle-compression': None},
 'array': {'svg': {'size': 120}}}

In [3]:
from dask_jobqueue import LSFCluster

In [4]:
# Per node specification
dask_worker_prefix = "jsrun -n1 -a1 -g0 -c1"

cluster = LSFCluster(
    scheduler_options={"dashboard_address": ":3762"},
    cores=8,
    processes=1, # default sqrt(cores). set to one to max threads per machine.  better for numpy.  see https://docs.dask.org/en/latest/setup/single-machine.html
    memory="4 GB",
    project="VEN201",
    walltime="00:10",
    job_extra=["-nnodes 1"],          # <--- new!
    header_skip=["-R", "-n ", "-M"],  # <--- new!
    interface='ib0',
    use_stdin=False,
)

## Lets See what is sent to LSF

In [5]:
print(cluster.job_script())

#!/usr/bin/env bash

#BSUB -J dask-worker
#BSUB -P VEN201
#BSUB -W 00:30
#BSUB -nnodes 1

/ccs/home/vanstee/.conda/envs/powerai-ornl/bin/python -m distributed.cli.dask_worker tcp://10.41.0.32:36525 --nthreads 8 --memory-limit 4.00GB --name name --nanny --death-timeout 60 --interface ib0



In [6]:
from dask.distributed import Client
client = Client(cluster)

In [6]:
# client.restart()

In [7]:
client

Client Scheduler: tcp://10.41.0.32:36525 Dashboard: http://10.41.0.32:3762/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [8]:
cluster.scale(2)

In [ ]:
client

In [12]:
!bjobs

JOBID   USER       STAT   SLOTS    QUEUE       START_TIME    FINISH_TIME   JOB_NAME                      
376497  vanstee    RUN    43       batch       Sep 29 16:23  Sep 29 16:53  dask-worker                   
376498  vanstee    RUN    43       batch       Sep 29 16:23  Sep 29 16:53  dask-worker                   


# Numpy simple example ...

In [14]:
import dask.array as da
# 2.5 B element array , 500 chunks
x = da.random.random([5000,5000], chunks=[250,250])


In [18]:
cluster.scale(8)

In [15]:
x = x.persist()
x

dask.array<random_sample, shape=(5000, 5000), dtype=float64, chunksize=(250, 250), chunktype=numpy.ndarray>

In [16]:
y = x.T ** x - x.mean()

In [17]:
y.persist()

dask.array<sub, shape=(5000, 5000), dtype=float64, chunksize=(250, 250), chunktype=numpy.ndarray>

In [38]:
#del(y)
y.compute()

AttributeError: 'str' object has no attribute 'shape'

In [ ]:
# Persist vs Compute https://distributed.dask.org/en/latest/memory.html
# use compute when the return value is small and you want to feed result into other analyses.
# use persist (similar to cache in spark) to trigger computation and pin results to memory.  
# Follow actions build task graphs, but only up to this point as it will use the value calculated by persist.

## Simple  Pandas Example with our lending club data ...

In [16]:
!ls ../Tabular/ldata2016.csv.gz

import dask
import dask.dataframe as dd
ddf = dd.read_csv("./ldata2016.csv", blocksize=15e6) # , compression="gzip")
#
#ddf = ddf.repartition(npartitions=5)
#ldata2016.csv.gz
ddf
print(ddf.columns)

../Tabular/ldata2016.csv.gz
Index(['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv',
       'term', 'int_rate', 'installment', 'grade', 'sub_grade',
       ...
       'hardship_payoff_balance_amount', 'hardship_last_payment_amount',
       'disbursement_method', 'debt_settlement_flag',
       'debt_settlement_flag_date', 'settlement_status', 'settlement_date',
       'settlement_amount', 'settlement_percentage', 'settlement_term'],
      dtype='object', length=151)


In [10]:
ddf.head()

/ccs/home/vanstee/.conda/envs/powerai-ornl/lib/python3.6/site-packages/dask/core.py:118: DtypeWarning: Columns (0,1,2,3,4,7,13,19,24,25,27,28,29,30,31,32,33,34,36,38,39,40,41,42,43,44,45,46,48,51,52,53,54,55,57,58,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,132,133,137,138,140,141,142,148,149,150) have mixed types. Specify dtype option on import or set low_memory=False.
  args2 = [_execute_task(a, cache) for a in args]


,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,Notes offered by Prospectus (https://www.lendingclub.com/info/prospectus.action)
id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
76003542,NaN,16000,16000,16000,36 months,5.32%,481.84,A,A1,Security specialist,8 years,RENT,105000,Not Verified,Mar-2016,Fully Paid,n,https://lendingclub.com/browse/loanDetail.action?loan_id=76003542,NaN,debt_consolidation,Debt consolidation,206xx,MD,15.02,1,Nov-2000,720,724,0,9,NaN,9,0,6219,27.9%,21,w,0.00,0.00,16098.34,16098.34,16000.00,98.34,0.0,0.0,0.0,May-2016,16107.8,NaN,Nov-2016,679,675,0,NaN,1,Individual,NaN,NaN,NaN,0,0,23525,0,3,2,2,8,17306,69,0,1,3770,50,22300,1,3,1,3,3361,13632,29.7,0,0,124,184,22,8,0,52,NaN,8,9,0,2,3,2,5,7,6,14,3,9,0,0,0,2,95.2,0,0,0,47543,23525,19400,25243,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
76023477,NaN,17000,17000,17000,36 months,9.75%,546.55,B,B3,Kindergarten Teacher,< 1 year,MORTGAGE,40000,Verified,Mar-2016,Current,n,https://lendingclub.com/browse/loanDetail.action?loan_id=76023477,NaN,credit_card,Credit card refinancing,440xx,NC,24.3,0,Aug-2006,705,709,0,NaN,NaN,4,0,14009,85.9%,8,w,1856.98,1856.98,16630.58,16630.58,15143.02,1487.56,0.0,0.0,0.0,Nov-2017,546.55,Dec-2017,Nov-2017,799,795,0,NaN,1,Individual,NaN,NaN,NaN,0,0,31939,0,1,0,0,25,17930,64,0,0,4913,72,16300,1,0,1,0,7985,2291,85.9,0,0,25,115,35,25,0,35,NaN,6,NaN,0,3,3,3,5,1,3,7,3,4,0,0,0,0,100,100,0,0,44131,31939,16300,27831,NaN,NaN,N

In [19]:
filtered_df = ddf["loan_amnt"] > 15000
filtered_df

Dask Series Structure:
npartitions=9
    bool
     ...
    ... 
     ...
     ...
Name: loan_amnt, dtype: bool
Dask Name: gt, 45 tasks

In [20]:
filtered_df.compute()

ValueError: Mismatched dtypes found in `pd.read_csv`/`pd.read_table`.

+----------------------------+---------+----------+
| Column                     | Found   | Expected |
+----------------------------+---------+----------+
| acc_now_delinq             | float64 | int64    |
| acc_open_past_24mths       | float64 | int64    |
| all_util                   | float64 | int64    |
| avg_cur_bal                | float64 | int64    |
| chargeoff_within_12_mths   | float64 | int64    |
| collections_12_mths_ex_med | float64 | int64    |
| delinq_2yrs                | float64 | int64    |
| delinq_amnt                | float64 | int64    |
| fico_range_high            | float64 | int64    |
| fico_range_low             | float64 | int64    |
| funded_amnt                | float64 | int64    |
| funded_amnt_inv            | float64 | int64    |
| id                         | object  | int64    |
| inq_fi                     | float64 | int64    |
| inq_last_12m               | float64 | int6

/ccs/home/vanstee/.conda/envs/powerai-ornl/lib/python3.6/site-packages/dask/core.py:118: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  args2 = [_execute_task(a, cache) for a in args]
